In [2]:
import sqlalchemy
import tool
user = "root"
password="root"
host="127.0.0.1"
database="text2sql"
engine = sqlalchemy.create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
s = tool.get_table_schema(engine)
s

'Table: mobile_data\n  Column: 日期, Type: DATE\n  Column: 网关SN, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 宽带账号, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 网关mac, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 中断总次数, Type: INTEGER\n  Column: 中断总时长(s), Type: INTEGER\n  Column: 厂商, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 型号, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 地市, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 区县, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 网格, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 小区名称, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 小区ID, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 覆盖方式, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 分纤箱, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: 鉴权码, Type: VARCHAR(255) COLLATE "utf8mb4_unicode_ci"\n  Column: PON口名称, Type: VARCHAR(255) COLLATE "utf8mb

In [3]:
import re
import sqlite3
from constants import DATABASE_PATH_PATTERN, DEV_JSON_PATH, TRAIN_JSON_PATH
import util
import json
import sqlparse
from thefuzz import process

with open(
    "/home/bingxing2/home/scx8900/projects/bird_train/train/train_tables.json", "r"
) as input_file:
    table_info = json.load(input_file)
comment_map = {}
for table in table_info:
    curr = {}
    table_names_original = table["table_names_original"]
    column_names = table["column_names"]
    column_names_original = table["column_names_original"]
    for i in range(len(column_names_original)):
        table_index = column_names_original[i][0]
        if table_index<0:
            continue
        table_name = table_names_original[table_index].lower()
        format_column_name = f"{table_name}.{column_names_original[i][1]}".lower()
        comment = column_names[i][1]
        curr[format_column_name] = comment
    comment_map[table["db_id"]] = curr

with open("bird_train_comment.json", "w") as output_file:
    json.dump(comment_map,output_file)

In [2]:
import json
with open("../bird_train/train/train.json", "r") as dataset_file:
    dataset = json.load(dataset_file)

In [4]:
import sqlite3
import re
import sqlite3
from constants import DATABASE_PATH_PATTERN, DEV_JSON_PATH, TRAIN_JSON_PATH
import util
import json
import sqlparse
from thefuzz import process

db_id = "pets_1"
db_path = DATABASE_PATH_PATTERN.format(db_id=db_id)
sqlite3.enable_callback_tracebacks(True)
try:
    
    conn = sqlite3.connect(db_path)

    # 创建一个游标对象
    cursor = conn.cursor()

    # 执行一个SQL语句
    cursor.execute('SELECT stuid FROM student EXCEPT SELECT t2.stuid FROM has_pet AS t1 JOIN pets AS t2 ON t1.petid = t2.petid WHERE t2.pettype = "cat";')

    # 获取查询结果
    rows = cursor.fetchall()

    # 打印结果
    for row in rows:
        print(row)


except sqlite3.Error as e:
    # 打印SQLite错误信息
    print(f"An error occurred: {e.args[0]}")
    # 如果有详细信息，打印详细信息
    # if len(e.args) > 1:
    print(f"Detailed error message: {e.args}")
    # 打印SQLite错误码和错误名称
    print(f"SQLite error code: {e.sqlite_errorcode}")
    print(f"SQLite error name: {e.sqlite_errorname}")

finally:
    # 关闭游标和数据库连接
    if cursor:
        cursor.close()
    if conn:
        conn.close()

An error occurred: no such column: t2.stuid
Detailed error message: ('no such column: t2.stuid',)
SQLite error code: 1
SQLite error name: SQLITE_ERROR


In [11]:
with open("bird_alpaca.json", "w") as f:
    json.dump(bird_dataset, f)

In [3]:

with open(TRAIN_JSON_PATH, "r") as dataset_file:
    dataset = json.load(dataset_file)


In [5]:
final_dataset = []
for case in dataset[:]:
    db_path = DATABASE_PATH_PATTERN.format(db_id=case["db_id"])
    schema = util.get_sqlite_alpaca_sft_prompt_schema(db_path)

    schema_str = ""
    for table, columns in schema.items():
        columns_str = ", ".join(columns)
        schema_str += f"{table}[{columns_str}]"
        schema_str += "\n"
    instruction = INSTRUCTION_PATTERN.format(
        schema=schema_str,
        question=case["question"],
    )
    sql = case["query"]
    sql = " ".join(sql.split())
    output = OUTPUT_PATTERN.format(sql=sql)
    # print(instruction)
    # print(output)
    final_dataset.append({"instruction": instruction, "output": output})
with open("spider_alpaca.json", "w") as f:
    json.dump(final_dataset, f)

In [2]:
import json
with open("spider_alpaca.json", "r") as dataset_file:
    spider = json.load(dataset_file)
with open("bird_alpaca.json", "r") as dataset_file:
    bird = json.load(dataset_file)
print(len(spider))
print(len(bird))

all_set = []
for case in spider:
    if len(case["instruction"]) < 10000:
        case["system"] = "You are a database expert.\n"
        all_set.append(case)
for case in bird:
    if len(case["instruction"]) < 10000:
        case["system"] = "You are a database expert.\n"
        all_set.append(case)
print(len(all_set))
with open("text2sql.json", "w") as f:
    json.dump(all_set, f)
    


8659
9428
17613


In [6]:
all_set = spider+bird_trunc
max_len=0
all_trunc = []
for case in all_set:
    if len(case["instruction"]) < 4000:
        all_trunc.append(case)
# print(all_trunc)
print(type(all_trunc))
print(len(all_trunc))

<class 'list'>
15535


In [7]:
with open("spider_bird.json", "w") as f:
    json.dump(all_trunc, f)

In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from constants import DATABASE_PATH_PATTERN, DEV_JSON_PATH
import util

print(torch.version.cuda)
model_path = "../deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


12.1


In [2]:
from prompt import SYSTEM_PROMPT


with open("cot_dataset/train/step_2.json", "r") as f:
    dataset = json.load(f)
total = 0 
for case in dataset:
    conversation = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": case["instruction"],
        },
        {"role": "assistant", "content": case["output"]},
    ]
    tokens = len(tokenizer.apply_chat_template(conversation, tokenize=True))
    total += tokens
print(total)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4551015
